In [59]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np
from tqdm import tqdm
import pandas as pd


import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import wandb

# local imports
from week1 import create_bag_visual_word, compute_feature_histogram
from cnn_models import get_CNN_model
from ml_models import get_ML_model
from utils import *

# GPU CHECK 

In [60]:
if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
    print("Tensorflow is not able to access the GPU !!")

Tensorflow is not able to access the GPU !!


# Constant values

In [61]:
IMG_SIZE = 256
IMG_CHANNEL = 3
BATCH_SIZE = 16
NUM_CLASSES = 8
NUM_EPOCH = 100

# data dir

In [62]:
train_dir = "/ghome/group05/mcv/datasets/C3/MIT_large_train/train"
test_dir = "/ghome/group05/mcv/datasets/C3/MIT_large_train/test"
validation_dir = "/ghome/group05/mcv/datasets/C3/MIT_small_train_2/test"

# DATASET PREAPRATION 

### augmentation

In [63]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,  # Degree range for random rotations
    # width_shift_range=0.2,  # Range (as a fraction of total width) for horizontal shift
    # height_shift_range=0.2,  # Range (as a fraction of total height) for vertical shift
    # shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction)
    # zoom_range=0.2,  # Range for random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    vertical_flip=True,
    fill_mode="nearest",  # Strategy for filling in newly created pixels
    rescale=1.0 / 255,  # normalize data
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,  # normalize data
)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,  # normalize data
)

### train, test and validation dataset


In [64]:
train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    classes=[
        "coast",
        "forest",
        "highway",
        "inside_city",
        "mountain",
        "Opencountry",
        "street",
        "tallbuilding",
    ],
    target_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    subset=None,
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    classes=[
        "coast",
        "forest",
        "highway",
        "inside_city",
        "mountain",
        "Opencountry",
        "street",
        "tallbuilding",
    ],
    target_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=123,
    subset=None,
)

validation_dataset = validation_datagen.flow_from_directory(
    directory=validation_dir,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    classes=[
        "coast",
        "forest",
        "highway",
        "inside_city",
        "mountain",
        "Opencountry",
        "street",
        "tallbuilding",
    ],
    target_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=123,
    subset=None,
)

Found 1881 images belonging to 8 classes.
Found 807 images belonging to 8 classes.
Found 2288 images belonging to 8 classes.


# early stopping

In [65]:
early_stopping_callback = CustomEarlyStoping(monitor="val_accuracy", threshold=0.95)

# train and test pipeline 

In [66]:
# helper function to get CNN feature vector and create feature and label for train and test
def extract_features(model, dataset):
    features = []
    labels = []
    count = 0
    for batch in tqdm(dataset, total=len(dataset)):
      
        images, batch_labels = batch

        batch_features = model.predict_on_batch(images)
        features.extend(batch_features)

        labels.extend(batch_labels)
        count += 1
        if count == len(dataset):
            break
    return np.array(features), np.array(labels)

In [67]:
def plot_image_grid(images, labels, predictions):
    # Set the number of images to display
    num_images = min(len(images), 10)  # For example, display up to 10 images

    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        plt.subplot(2, num_images // 2, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(f"True: {labels[i]}\nPred: {predictions[i]}")
    plt.show()

In [68]:
def evaluate_model(predicted_labels, test_features, test_labels, test_dataset):
    # predictions = model.predict(test_features)
    # predicted_labels = np.argmax(predictions, axis=1)

    # Confusion Matrix
    cm = confusion_matrix(test_labels, predicted_labels)
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("Actual Labels")
    plt.xlabel("Predicted Labels")
    plt.show()

    # Precision, Recall, F1-Score
    print(classification_report(test_labels, predicted_labels))

    # Plotting correctly and wrongly predicted images
    correct = np.where(predicted_labels == test_labels)[0]
    incorrect = np.where(predicted_labels != test_labels)[0]
    plot_image_grid(
        test_dataset[correct], test_labels[correct], predicted_labels[correct]
    )
    plot_image_grid(
        test_dataset[incorrect], test_labels[incorrect], predicted_labels[incorrect]
    )

In [69]:
def train_test(
    TRAINING_METHOD: str,
    CNN_MODEL: str,
    ML_cLASSIFIER_MODEL: str = None,
    NUMER_PATCH: int = None,
):
    if TRAINING_METHOD == "END_TO_END_CNN":
        model = get_CNN_model(
            MODEL_NAME=CNN_MODEL,
            IMG_SIZE=IMG_SIZE,
            IMG_CHANNEL=IMG_CHANNEL,
            NUM_CLASSES=NUM_CLASSES,
        )
        model.compile(
            optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
        )

        history = model.fit(
            train_dataset,
            epochs=NUM_EPOCH,
            validation_data=validation_dataset,
            callbacks=[early_stopping_callback],
        )
        test_loss, test_accuracy = model.evaluate(test_dataset)

        return test_accuracy
    elif TRAINING_METHOD == "CNN_FEATURE_WITH_ML_CLASSIFIER":
        cnn_model = get_CNN_model(
            MODEL_NAME=CNN_MODEL,
            IMG_SIZE=IMG_SIZE,
            IMG_CHANNEL=IMG_CHANNEL,
            NUM_CLASSES=NUM_CLASSES,
            ONLY_FEATURE=True,
        )

        train_features, train_labels = extract_features(cnn_model, train_dataset)
        train_features = np.array(train_features)
        train_labels = np.array(train_labels)
        train_labels = np.argmax(train_labels, axis=1)


        test_features, test_labels = extract_features(cnn_model, train_dataset)
        test_features = np.array(test_features)
        test_labels = np.array(test_labels)
        test_labels = np.argmax(test_labels, axis=1)



        ml_classifier = get_ML_model(
            MODEL_NAME=ML_cLASSIFIER_MODEL, train_X=train_features, train_y=train_labels
        )
        test_accuracy = ml_classifier.score(test_features, test_labels)
        
        
        test_pred = ml_classifier.predict(test_features)
     

        # 3. Create Confusion Matrix
        conf_matrix = confusion_matrix(test_labels, test_pred)

        # 4. Get Class Indices to Names Map
        class_indices_to_names = {v: k for k, v in test_dataset.class_indices.items()}

        # 5. Adjust Confusion Matrix with Class Names
        conf_matrix_with_names = pd.DataFrame(
            conf_matrix,
            index=[class_indices_to_names[i] for i in range(NUM_CLASSES)],
            columns=[class_indices_to_names[i] for i in range(NUM_CLASSES)],
        )

        # 6. Plot Confusion Matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix_with_names, annot=True, fmt="d", cmap="Blues")
        plt.title("Confusion Matrix")
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        plt.show()
        
        
        test_labels = [class_indices_to_names[i] for i in test_labels]
        test_pred = [class_indices_to_names[i] for i in test_pred]
        print(classification_report(test_labels, test_pred))

      
        return test_accuracy
    elif TRAINING_METHOD == "CNN_FEATURE_WITH_BOVW":
        cnn_model = get_CNN_model(
            MODEL_NAME=CNN_MODEL,
            IMG_SIZE=IMG_SIZE,
            IMG_CHANNEL=IMG_CHANNEL,
            NUM_CLASSES=NUM_CLASSES,
            ONLY_FEATURE=True,
        )

        train_features, train_labels = extract_features(cnn_model, train_dataset)
        train_features = np.array(train_features)
        train_labels = np.array(train_labels)
        train_labels = np.argmax(train_labels, axis=1)
        print(f"train_features shape: {train_features.shape}")
        print(f"train_labels shape  : {train_labels.shape}")

        test_features, test_labels = extract_features(cnn_model, train_dataset)
        test_features = np.array(test_features)
        test_labels = np.array(test_labels)
        test_labels = np.argmax(test_labels, axis=1)

        print(f"test_features shape: {test_features.shape}")
        print(f"test_labels shape  : {test_labels.shape}")

        codebook = create_bag_visual_word(train_features, 5)

        train_y = []
        train_X = []

        for features, label in zip(train_features, train_labels):
            res = compute_feature_histogram(
                features=features.reshape(1, -1), codebook=codebook
            )

            train_X.append(res)
            train_y.append(label)

        test_X = []
        test_y = []

        for features, label in zip(test_features, test_labels):
            res = compute_feature_histogram(
                features=features.reshape(1, -1), codebook=codebook
            )

            test_X.append(res)
            test_y.append(label)

        ml_classifier = get_ML_model(
            MODEL_NAME=ML_cLASSIFIER_MODEL, train_X=train_X, train_y=train_y
        )

        test_accuracy = ml_classifier.score(test_X, test_y)
        evaluate_model()
        print(f"ACCURACY: {test_accuracy}")
        return test_accuracy
    elif TRAINING_METHOD == "KEYPOINT_FEATURE_wITH_CNN":
        return

# optuna and weight & bias setup

In [70]:
wandb_kwargs = {"project": "WEEK_3_FINDING_BEST_COMBINATIN"}
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, as_multirun=True)

In [71]:
available_training_method = [
    "END_TO_END_CNN",
    "CNN_FEATURE_WITH_ML_CLASSIFIER",
    "CNN_FEATURE_WITH_BOVW" "KEYPOINT_FEATURE_wITH_CNN",
]
available_CNN_models = [
    "pretrained_mobilenet_V2",
    "pretrained_resnet50",
    "pretrained_efficientnet",
    "pretrained_vgg16",
    "pretrained_vit",
    "pretrained_densenet",

    "scratch_CNN_Residual_Model",
    "scratch_CNN_Patch_Model",
]
available_ML_classifier_models = [
    "SVM",
    "KNN",
    "XGBOOST",
    "DecisionTree",
    "LogisticRegression",
]

In [72]:
def objective(trial):
    wandb.init(project="WEEK3_testing")
    selected_training_method = trial.suggest_categorical(
        "training_method", available_training_method
    )

    selected_CNN_model = trial.suggest_categorical("CNN_model", available_CNN_models)
    if selected_CNN_model == "CNN_Patch_Model":
        seleted_patch_size = trial.suggest_categorical(
            "patch_size", [i - 1 for i in range(1, 100, 5)]
        )
        pass

    else:
        seleted_patch_size = None

    if (
        selected_training_method == "CNN_FEATURE_WITH_ML_CLASSIFIER"
        or selected_training_method == "CNN_FEATURE_WITH_BOVW"
    ):
        selected_ML_classifier_model = trial.suggest_categorical(
            "ML_classifier_models", available_ML_classifier_models
        )

    else:
        selected_ML_classifier_model = None

    accuracy = train_test(
        TRAINING_METHOD=selected_training_method,
        CNN_MODEL=selected_CNN_model,
        ML_cLASSIFIER_MODEL=selected_ML_classifier_model,
        NUMER_PATCH = seleted_patch_size
        
    )

    wandb.log(
        {
            "accuracy": accuracy,
        }
    )
    return accuracy

In [73]:
# if __name__ == "__main__":
#     study = optuna.create_study(
#         direction="maximize",
#         storage="sqlite:///db.sqlite3",
#         pruner=optuna.pruners.SuccessiveHalvingPruner(),
#     )
#     study.optimize(
#         objective,
#         n_trials=1000,
#         timeout=60000000000000000000,
#         callbacks=[wandbc],  # weight and bias connection
#     )

# model performance testing

In [74]:
TRAINING_METHOD = "CNN_FEATURE_WITH_ML_CLASSIFIER"
CNN_MODEL = "pretrained_mobilenet_V2"
ML_cLASSIFIER_MODEL = "LogisticRegression"
NUMER_PATCH = 0


accuracy = train_test(
    TRAINING_METHOD=TRAINING_METHOD,
    CNN_MODEL=CNN_MODEL,
    ML_cLASSIFIER_MODEL=ML_cLASSIFIER_MODEL,
    NUMER_PATCH=NUMER_PATCH,
)

  5%|▌         | 6/118 [00:02<00:40,  2.78it/s]

# cross validation